# BUILDING DEEP LEARNING APPLICATIONS WITH KERAS 2.0

## Preprocessing training data

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv")

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv")

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new pandas DataFrame objects from the scaled data
scaled_training_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

# Save scaled data dataframes to new CSV files
scaled_training_df.to_csv("sales_data_training_scaled.csv", index=False)
scaled_testing_df.to_csv("sales_data_testing_scaled.csv", index=False)

Note: total_earnings values were scaled by multiplying by 0.0000036968 and adding -0.115913


## Neural Network tensorflow

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

#training_data_df = pd.read_csv(io.BytesIO(uploaded4["sales_data_training_scaled.csv"]))
training_data_df = pd.read_csv('sales_data_training_scaled.csv')

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

# Load the separate test data set
#test_data_df = pd.read_csv(io.BytesIO(uploaded5['sales_data_test_scaled.csv']))
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

# Save the model to disk
model.save("trained_model.h5")
print("Model saved to disk.")


Epoch 1/50
32/32 - 4s - loss: 0.0042 - 4s/epoch - 127ms/step
Epoch 2/50
32/32 - 0s - loss: 7.4114e-04 - 79ms/epoch - 2ms/step
Epoch 3/50
32/32 - 0s - loss: 4.6496e-04 - 59ms/epoch - 2ms/step
Epoch 4/50
32/32 - 0s - loss: 2.4523e-04 - 56ms/epoch - 2ms/step
Epoch 5/50
32/32 - 0s - loss: 1.5961e-04 - 74ms/epoch - 2ms/step
Epoch 6/50
32/32 - 0s - loss: 1.3502e-04 - 74ms/epoch - 2ms/step
Epoch 7/50
32/32 - 0s - loss: 9.7302e-05 - 84ms/epoch - 3ms/step
Epoch 8/50
32/32 - 0s - loss: 9.0333e-05 - 72ms/epoch - 2ms/step
Epoch 9/50
32/32 - 0s - loss: 1.1425e-04 - 68ms/epoch - 2ms/step
Epoch 10/50
32/32 - 0s - loss: 9.9293e-05 - 69ms/epoch - 2ms/step
Epoch 11/50
32/32 - 0s - loss: 7.4203e-05 - 68ms/epoch - 2ms/step
Epoch 12/50
32/32 - 0s - loss: 6.0192e-05 - 67ms/epoch - 2ms/step
Epoch 13/50
32/32 - 0s - loss: 8.4150e-05 - 83ms/epoch - 3ms/step
Epoch 14/50
32/32 - 0s - loss: 9.0512e-05 - 79ms/epoch - 2ms/step
Epoch 15/50
32/32 - 0s - loss: 1.3868e-04 - 101ms/epoch - 3ms/step
Epoch 16/50
32/32 - 0s

## Load saved model


In [3]:
import pandas as pd
from keras.models import load_model

model = load_model('trained_model.h5')

X = pd.read_csv("proposed_new_product.csv").values
prediction = model.predict(X)

# Grab just the first element of the first prediction (since we only have one)
prediction = prediction[0][0]

# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = prediction + 0.1159
prediction = prediction / 0.0000036968

print("Earnings Prediction for Proposed Product - ${}".format(prediction))



Earnings Prediction for Proposed Product - $263765.7144321969


## Monitoring a Keras model with TensorBoard

In [4]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir='logs',
    write_graph=True,
    histogram_freq=5
)

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/50
32/32 - 2s - loss: 0.0423 - 2s/epoch - 52ms/step
Epoch 2/50
32/32 - 0s - loss: 0.0045 - 98ms/epoch - 3ms/step
Epoch 3/50
32/32 - 0s - loss: 0.0011 - 100ms/epoch - 3ms/step
Epoch 4/50
32/32 - 0s - loss: 5.2845e-04 - 101ms/epoch - 3ms/step
Epoch 5/50
32/32 - 0s - loss: 3.1292e-04 - 99ms/epoch - 3ms/step
Epoch 6/50
32/32 - 0s - loss: 2.2237e-04 - 178ms/epoch - 6ms/step
Epoch 7/50
32/32 - 0s - loss: 1.6727e-04 - 153ms/epoch - 5ms/step
Epoch 8/50
32/32 - 0s - loss: 1.4028e-04 - 103ms/epoch - 3ms/step
Epoch 9/50
32/32 - 0s - loss: 1.1762e-04 - 99ms/epoch - 3ms/step
Epoch 10/50
32/32 - 0s - loss: 9.5121e-05 - 102ms/epoch - 3ms/step
Epoch 11/50
32/32 - 0s - loss: 8.3845e-05 - 195ms/epoch - 6ms/step
Epoch 12/50
32/32 - 0s - loss: 6.9134e-05 - 89ms/epoch - 3ms/step
Epoch 13/50
32/32 - 0s - loss: 7.0336e-05 - 116ms/epoch - 4ms/step
Epoch 14/50
32/32 - 0s - loss: 5.6315e-05 - 99ms/epoch - 3ms/step
Epoch 15/50
32/32 - 0s - loss: 5.1815e-05 - 102ms/epoch - 3ms/step
Epoch 16/50
32/32 - 0s 

In [3]:
from platform import python_version
print(python_version())

3.8.12


In [4]:
import tensorflow as tf
print(tf.__version__)

2.8.0
